# Algorithm2: Shop dictionary processing

In algorithm 2, we propose the code to process the shop dictionary, define the data type of the key, generate the string values and double values and key diversity , key dimension, and std.

In [ ]:
# First, convert the json into python dictionary
import json
    
## read the json file
f = open('C:\\\\Users\\\\mingh\\\\Desktop\\\\text\\\\Master\\\\Thesis\\\\Python Code\\\\Algorithm 1\\\\shop_dict.json')
data = json.load(f)

In [ ]:
# here we define the common unit in the product festures
common_unit = ['inches','"','lbs','lb','kg','cd/m','Nit','$','pounds','mm','ms','W','V']

Here we defind the function to gethering the dimension of double type keys.

In [3]:
# define the frequency function to calculate the data dimension
def most_frequent(List):
    return max(set(List), key = List.count)

In the folloing block, we detect the key type, the string and double values, the key dimension and diversity.

In [5]:
import re

# figure out the key type is str or double
for shop in data.values():
    for key in shop['product_detail']['featuresMap'].keys():
        valuetype = {'string':0,'double':0}
        doubleValues = []
        dimension = []
        for i in range(len(shop['product_detail']['featuresMap'][key])):
            # select each value, extract its digit element
            value = shop['product_detail']['featuresMap'][key][i]
            numbers = [float(s) for s in re.findall(r'\d+\.\d+|\d+', value)]
            doubleValues.append(numbers)
            dimension.append(len(numbers))
            
            # define the lenth of value
            value_len = len(value)
            
            # define the length of digit element
            num_len = 0
            for i in range(len(numbers)):
                num_len += len(str(numbers[i]))
                
            # define the data type of each element
            if num_len > value_len*0.5:
                valuetype['double'] += 1 
            elif any(unit in value for unit in common_unit):
                valuetype['double'] += 1
            else: valuetype['string'] += 1 
        
        # now compare the count of valuetype
        if valuetype['double'] >= valuetype['string']:
            keytype = 'double'
        else :
            keytype = 'string'
            
        # prepare the unique value list
        rawValues = list(shop['product_detail']['featuresMap'][key])
        if keytype == 'string':
            stringValues = list(shop['product_detail']['featuresMap'][key])
            doubleValues = 'None'
            dimension = 'None'
        else:
            stringValues = 'None'
            dimension = most_frequent(dimension)
        
        diversity = len(shop['product_detail']['featuresMap'][key])
            
        # reform the key matrix    
        shop['product_detail']['featuresMap'][key] = dict({'keytype':keytype,'Raw Value':rawValues,'String Values':stringValues,'Double Values':doubleValues,'Dimension':dimension,'Diversity':diversity})
    

In [6]:
# remove the block double data 

for shop in data.values():
    for key in shop['product_detail']['featuresMap'].values():
        if key['keytype'] == 'double':
            key['Double Values'] = [x for x in key['Double Values'] if x != []]
        else: pass
                

Now we gethering the Std of the double type keys.

In [8]:
# get std for all double key
import numpy as np

for shop in data.values():
    for key in shop['product_detail']['featuresMap'].keys():
        # if not double value the pass
        if shop['product_detail']['featuresMap'][key]['keytype'] == 'string':
            pass
        # select all the double value key
        else:
            dimension = shop['product_detail']['featuresMap'][key]['Dimension']
            std = []
            
            # select all the product value only in the same dimension with the key
            for numbers in shop['product_detail']['featuresMap'][key]['Double Values']:
                if len(numbers) == dimension:
                    std.append(numbers)
                else: pass
            
            if std == []:
                std = ['None']
            if len(std[0]) > 1:
                std_in = []
                position = 0
                while position < dimension:
                    #creat the parameter
                    number_in = []
                    
                    for i in std:
                        number_in.append(i[position])
                    
                    std_in.append(np.std(number_in))
                    position += 1
                    
                Standard_Deviation = list(std_in)
            elif std[0] == 'None':
                Standard_Deviation = 'None'
                    
            else: 
                Standard_Deviation = np.std(std)
            
            shop['product_detail']['featuresMap'][key]['std'] = Standard_Deviation

In the end, we clean and export the data.

In [9]:
output = {}
for shop in data.keys():
    output[shop] = data[shop]['product_detail']['featuresMap']

In [10]:
import json
with open('shop_dict_A2.json', 'w') as f:
    json.dump(output, f)